# Assignment 4

In this assignment, you'll combine the assignment 3 data set with nutrition data from the [USDA Food Composition Databases](https://ndb.nal.usda.gov/ndb/search/list). The CSV file `fresh.csv` contains the fresh fruits and vegetables data you extracted in assignment 3.

The USDA Food Composition Databases have a [documented](https://ndb.nal.usda.gov/ndb/doc/index) web API that returns data in JSON format . You need a key in order to use the API. Only 1000 requests are allowed per hour, so it would be a good idea to use [caching][requests_cache].

[Sign up for an API key here](https://api.data.gov/signup/). The key will work with any Data.gov API. You may need the key again later in the quarter, so make sure you save it.

These modules may be useful:

* [requests](http://docs.python-requests.org/en/master/user/quickstart/)
* [requests_cache][]
* [urlparse](https://docs.python.org/2/library/urlparse.html)
* [pandas](http://pandas.pydata.org/pandas-docs/stable/)

[requests_cache]: https://pypi.python.org/pypi/requests-cache

__Exercise 1.1.__ Read the [search request documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-SEARCH.md), then write a function called `ndb_search()` that makes a search request. The function should accept the search term as an argument. The function should return the search result items as a list (for 0 items, return an empty list).

Note that the search url is: `https://api.nal.usda.gov/ndb/search`

As an example, a search for `"quail eggs"` should return this list:

```python
[{u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  u'ndbno': u'45094707',
  u'offset': 0},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'L&W, QUAIL EGGS, UPC: 024072000256',
  u'ndbno': u'45094890',
  u'offset': 1},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  u'ndbno': u'45099560',
  u'offset': 2},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  u'ndbno': u'45169279',
  u'offset': 3},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u"D'ARTAGNAN, QUAIL EGGS, UPC: 736622102630",
  u'ndbno': u'45178254',
  u'offset': 4},
 {u'ds': u'SR',
  u'group': u'Dairy and Egg Products',
  u'name': u'Egg, quail, whole, fresh, raw',
  u'ndbno': u'01140',
  u'offset': 5}]
```

As usual, make sure you document and test your function.

In [427]:
from urllib2 import Request, urlopen
from urlparse import urlparse, urlunparse
import requests, requests_cache
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
requests_cache.install_cache('cache')

In [263]:
def ndb_search(term,ds):
    """
    Input:
    Output: 
    """
    term = term.replace("_", " ")
    url = "https://api.nal.usda.gov/ndb/search"
    response = requests.get(url, params = {
        "q" : term,
        "api_key" : "x1GQ2744tFjXxOhyCMtQTQeObIitFUAw3FPCwwBF",
        "format" : "JSON",
        "ds" : ds
    })
    response.raise_for_status()
    if "list" in response.json():
        result = response.json()["list"]["item"]
    else:
        result = []
    return result 

In [89]:
ndb_search("quail eggs", ds = "")

[{u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  u'ndbno': u'45094707',
  u'offset': 0},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'L&W, QUAIL EGGS, UPC: 024072000256',
  u'ndbno': u'45094890',
  u'offset': 1},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  u'ndbno': u'45099560',
  u'offset': 2},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  u'ndbno': u'45169279',
  u'offset': 3},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u"D'ARTAGNAN, QUAIL EGGS, UPC: 736622102630",
  u'ndbno': u'45178254',
  u'offset': 4},
 {u'ds': u'SR',
  u'group': u'Dairy and Egg Products',
  u'name': u'Egg, quail, whole, fresh, raw',
  u'ndbno': u'01140',
  u'offset': 5}]

In [55]:
ndb_search("hkjfsdhkf")

[]

__Exercise 1.2.__ Use your search function to get NDB numbers for the foods in the `fresh.csv` file. It's okay if you don't get an NDB number for every food, but try to come up with a strategy that gets most of them. Discuss your strategy in a short paragraph.

Hints:

* The foods are all raw and unbranded.
* You can test search terms with the [online search page](https://ndb.nal.usda.gov/ndb/search/list).
* You can convert the output of `ndb_search()` to a data frame with `pd.DataFrame()`.
* The string methods for [Python](https://docs.python.org/2/library/stdtypes.html#string-methods) and [Pandas](http://pandas.pydata.org/pandas-docs/stable/text.html#method-summary) are useful here. It's okay if you use _simple_ regular expressions in the Pandas methods, although this exercise can be solved without them.
* You can merge data frames that have a column in common with `pd.merge()`.

In [502]:
fresh = pd.read_csv('G:/study/2016-2017/winter/141b/hw/hw4/fresh.csv',header = 0)
fresh["form"][20] = fresh["form"][20].strip()

d:\python27\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [522]:
def choose(item):
    if item != "collard_greens":
        original = pd.DataFrame(ndb_search(item , ds = "Standard Reference"))
    else:
        original = pd.DataFrame(ndb_search("collard" , ds = "Standard Reference"))
        
    original["food"] = item 
    raw = original[original.name.str.contains(" raw", case = False)&~original.name.str.contains(" juice", case = False)]
    raw_group = raw[raw.group.str.contains("Vegetable|Fruit")]
    return raw_group

In [504]:
def ndb_final(item):
    if sum(fresh[fresh.food == item]["form"] == "Fresh1") >= 1:
        raw_group = choose(item)
        if raw_group.shape[0] == 1:
            return raw_group
        if raw_group.shape[0] > 1:
            return raw_group[raw_group.ndbno == min(raw_group.ndbno)]
    else:
        original = choose(item)
        a = fresh[fresh.food == item]["form"]
        import re #for regular expression
        h = [re.findall(r"Fresh(.+?)1", x)[0].split(" ")[-2:] for x in a]
        flag = [np.logical_and(original.name.str.contains(" "+ x[0].replace(",",""), case = False),original.name.str.contains(x[1], case = False)) for x in h]
        n = len(flag)
        result = original[flag[0]]
        for i in range(n-1):
            result = pd.merge(result, original[flag[i+1]], how = "outer") 
        return result

In [537]:
n = len(fresh["food"])
b = ndb_final(fresh["food"][0])
rr=[]
for i in range(n):
    rr.append(ndb_final(fresh["food"][i]))

myrr=pd.concat(rr)

In [538]:
myrr

,ds,group,name,ndbno,offset,food
0,SR,Fruits and Fruit Juices,"Watermelon, raw",09326,0,watermelon
0,SR,Fruits and Fruit Juices,"Melons, cantaloupe, raw",09181,0,cantaloupe
0,SR,Fruits and Fruit Juices,"Tangerines, (mandarin oranges), raw",09218,0,tangerines
5,SR,Fruits and Fruit Juices,"Guavas, strawberry, raw",09140,5,strawberries
2,SR,Fruits and Fruit Juices,"Carissa, (natal-plum), raw",09061,2,plums
25,SR,Fruits and Fruit Juices,"Oranges, raw, all commercial varieties",09200,25,oranges
0,SR,Fruits and Fruit Juices,"Raspberries, raw",09302,0,raspberries
0,SR,Fruits and Fruit Juices,"Pomegranates, raw",09286,0,pomegranate
1,SR,Fruits and Fruit Juices,"Pineapple, raw, all varieties",09266,1,pineapple
0,SR,Fruits and Fruit Juices,"Apricots, raw",09021,0,apricots


1. Set ds as "Standard Reference"
2. Check if the original names contain " raw"
3. After looking at the table I found the original table contains
   
   1 	SR 	Fruits and Fruit Juices 	Tangerine juice, raw 	09221 	1
   
   22 	SR 	Finfish and Shellfish Products 	Fish, roughy, orange, raw 	15073 	22
   
   So we need to choose name without juice. So we need add some constrains for group.
4. for cucumber and cabbage I choose to find their form and make find the

__Exercise 1.3.__ Read the [food reports V2 documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-FOOD-REPORTV2.md), then write a function called `ndb_report()` that requests a _basic_ food report. The function should accept the NDB number as an argument and return the list of nutrients for the food.

Note that the report url is: `https://api.nal.usda.gov/ndb/V2/reports`

For example, for `"09279"` (raw plums) the first element of the returned list should be:

```python
{u'group': u'Proximates',
 u'measures': [{u'eqv': 165.0,
   u'eunit': u'g',
   u'label': u'cup, sliced',
   u'qty': 1.0,
   u'value': u'143.93'},
  {u'eqv': 66.0,
   u'eunit': u'g',
   u'label': u'fruit (2-1/8" dia)',
   u'qty': 1.0,
   u'value': u'57.57'},
  {u'eqv': 151.0,
   u'eunit': u'g',
   u'label': u'NLEA serving',
   u'qty': 1.0,
   u'value': u'131.72'}],
 u'name': u'Water',
 u'nutrient_id': u'255',
 u'unit': u'g',
 u'value': u'87.23'}
```

Be sure to document and test your function.

In [145]:
def ndb_report(ndb_num):
    """
    Input:
    Output: j
    """
    url = "https://api.nal.usda.gov/ndb/V2/reports"
    response = requests.get(url, params = {
        "ndbno" : ndb_num,
        "api_key" : "x1GQ2744tFjXxOhyCMtQTQeObIitFUAw3FPCwwBF",
        "format" : "JSON"
    })
    response.raise_for_status()
    result = response.json()['foods'][0]['food']['nutrients']
    return result 

In [147]:
ndb_report("09279")

[{u'group': u'Proximates',
  u'measures': [{u'eqv': 165.0,
    u'eunit': u'g',
    u'label': u'cup, sliced',
    u'qty': 1.0,
    u'value': u'143.93'},
   {u'eqv': 66.0,
    u'eunit': u'g',
    u'label': u'fruit (2-1/8" dia)',
    u'qty': 1.0,
    u'value': u'57.57'},
   {u'eqv': 151.0,
    u'eunit': u'g',
    u'label': u'NLEA serving',
    u'qty': 1.0,
    u'value': u'131.72'}],
  u'name': u'Water',
  u'nutrient_id': u'255',
  u'unit': u'g',
  u'value': u'87.23'},
 {u'group': u'Proximates',
  u'measures': [{u'eqv': 165.0,
    u'eunit': u'g',
    u'label': u'cup, sliced',
    u'qty': 1.0,
    u'value': u'76'},
   {u'eqv': 66.0,
    u'eunit': u'g',
    u'label': u'fruit (2-1/8" dia)',
    u'qty': 1.0,
    u'value': u'30'},
   {u'eqv': 151.0,
    u'eunit': u'g',
    u'label': u'NLEA serving',
    u'qty': 1.0,
    u'value': u'69'}],
  u'name': u'Energy',
  u'nutrient_id': u'208',
  u'unit': u'kcal',
  u'value': u'46'},
 {u'group': u'Proximates',
  u'measures': [{u'eqv': 165.0,
    u'eunit

__Exercise 1.4.__ Which foods provide the best combination of price, yield, and nutrition? You can use kilocalories as a measure of "nutrition" here, but more a detailed analysis is better. Use plots to support your analysis.

In [ ]:
fresh['nutrition']=''
for x in myrr.index:
    num=myrr.ix[x,'ndbno']
    if num!='':
        r=ndb_report(num)[1]['value']
        fresh.ix[x,'nutrition']=r